# data cleaning

In [16]:
import pandas as pd
import numpy as np


# read the country_vaccinations.csv file
vacc_df = pd.read_csv('https://raw.githubusercontent.com/jfoo0016/FIT3179_Assignment2/main/data/country_vaccinations.csv')

# read the worldometer_coronavirus_summary_data.csv file
summary_df = pd.read_csv('https://raw.githubusercontent.com/jfoo0016/FIT3179_Assignment2/main/data/worldometer_coronavirus_summary_data.csv')

In [17]:
# standardize country names

# identify the differences
print("Countries in the Vaccination Data not in Summary Data")
print([x for x in vacc_df.country.unique() if x not in summary_df.country.unique()])

Countries in the Vaccination Data not in Summary Data
['Antigua and Barbuda', 'Bonaire Sint Eustatius and Saba', 'Bosnia and Herzegovina', 'Cape Verde', 'Central African Republic', "Cote d'Ivoire", 'Czechia', 'Democratic Republic of Congo', 'Dominican Republic', 'England', 'Equatorial Guinea', 'Eswatini', 'Falkland Islands', 'Guernsey', 'Hong Kong', 'Isle of Man', 'Jersey', 'Kosovo', 'Macao', 'North Macedonia', 'Northern Cyprus', 'Northern Ireland', 'Pitcairn', 'Saint Kitts and Nevis', 'Saint Vincent and the Grenadines', 'Sao Tome and Principe', 'Scotland', 'Sint Maarten (Dutch part)', 'Solomon Islands', 'South Sudan', 'Timor', 'Tokelau', 'Turkmenistan', 'Turks and Caicos Islands', 'Tuvalu', 'United States', 'Wales', 'Wallis and Futuna']


In [18]:
# print out the list of country names in summary data for easier reference
print(summary_df.country.unique().tolist())

['Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola', 'Anguilla', 'Antigua And Barbuda', 'Argentina', 'Armenia', 'Aruba', 'Australia', 'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain', 'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bermuda', 'Bhutan', 'Bolivia', 'Bosnia and Herz.', 'Botswana', 'Brazil', 'British Virgin Islands', 'Brunei', 'Bulgaria', 'Burkina Faso', 'Burundi', 'Cabo Verde', 'Cambodia', 'Cameroon', 'Canada', 'Caribbean Netherlands', 'Cayman Islands', 'Central African Rep.', 'Chad', 'Channel Islands', 'Chile', 'China Hong Kong Sar', 'China Macao Sar', 'China', 'Colombia', 'Comoros', 'Congo', 'Cook Islands', 'Costa Rica', "Côte d'Ivoire", 'Croatia', 'Cuba', 'Curacao', 'Cyprus', 'Czech Republic', 'Dem. Rep. Congo', 'Denmark', 'Djibouti', 'Dominica', 'Dominican Rep.', 'Ecuador', 'Egypt', 'El Salvador', 'Eq. Guinea', 'Eritrea', 'Estonia', 'Ethiopia', 'Faeroe Islands', 'Falkland Is.', 'Fiji', 'Finland', 'France', 'French Guiana', 'French Polynesia', 'Ga

In [19]:
# replace the names in vacc_df as the names in summary_df is already following the naming of countries from natural earth dataset
vacc_df.country = vacc_df.country.replace().replace({
    "Antigua and Barbuda": "Antigua and Barbuda",
    "Bosnia and Herzegovina": "Bosnia and Herz.",
    "Central African Republic": "Central African Rep.",
    "Cote d'Ivoire": "Côte d'Ivoire",
    "Czechia": "Czech Republic",
    "Democratic Republic of Congo": "Dem. Rep. Congo", 
    "Dominican Republic": "Dominican Rep.",
    "Equatorial Guinea": "Eq. Guinea",
    "Falkland Islands": "Falkland Is.",
    "Hong Kong": "China Hong Kong Sar",
    "Macao": "China Macao Sar",
    "United States": "United States of America", 
    "Isle of Man": "Isle Of Man",
    "Republic of Ireland": "Ireland",
    "Northern Cyprus" : "Cyprus",
    "Solomon Islands": "Solomon Is.",
    "South Sudan": "S. Sudan",
    "Timor": "Timor-Leste"
})

# drop these 3 since they are included in UK 
vacc_df = vacc_df[vacc_df.country.apply(lambda x: x not in ['England', 'Scotland', 'Wales', 'Northern Ireland'])]

In [20]:
# function to easily agrregate columns
def aggregate(df: pd.Series, agg_col: str) -> pd.DataFrame:
    
    data = df.groupby("country")[agg_col].max()
    data = pd.DataFrame(data)
    
    return data

In [21]:
# define the columns we want to summarize
cols_to_summarize = ['people_vaccinated', 
                     'people_vaccinated_per_hundred', 
                     'people_fully_vaccinated', 
                     'people_fully_vaccinated_per_hundred', 
                     'total_vaccinations_per_hundred', 
                     'total_vaccinations']

summary = summary_df.set_index("country")
vaccines = vacc_df[['country', 'vaccines']].drop_duplicates().set_index('country')
summary = summary.join(vaccines)

for col in cols_to_summarize:   
    summary = summary.join(aggregate(vacc_df, col))

summary['percentage_vaccinated'] = summary.total_vaccinations / summary.population * 100
summary['tested_positive'] = summary.total_confirmed / summary.total_tests * 100
summary['percentage_fully_vaccinated'] = summary.people_fully_vaccinated / summary.population * 100

In [22]:
summary.head()

,Unnamed: 0,continent,total_confirmed,total_deaths,total_recovered,active_cases,serious_or_critical,total_cases_per_1m_population,total_deaths_per_1m_population,total_tests,...,vaccines,people_vaccinated,people_vaccinated_per_hundred,people_fully_vaccinated,people_fully_vaccinated_per_hundred,total_vaccinations_per_hundred,total_vaccinations,percentage_vaccinated,tested_positive,percentage_fully_vaccinated
country,,,,,,,,,,,,,,,,,,,,,
Afghanistan,0,Asia,179267,7690.0,162202.0,9375.0,1124.0,4420,190.0,951337.0,...,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi...",5082824.0,12.76,4420127.0,11.10,14.44,5751015.0,14.178809,18.843691,10.897578
Albania,1,Europe,275574,3497.0,271826.0,251.0,2.0,95954,1218.0,1817530.0,...,"Oxford/AstraZeneca, Pfizer/BioNTech, Sinovac, ...",1278902.0,44.52,1215199.0,42.30,95.87,2754244.0,95.901697,15.162006,42.312753
Algeria,2,Africa,265816,6875.0,178371.0,80570.0,6.0,5865,152.0,230861.0,...,"Oxford/AstraZeneca, Sinopharm/Beijing, Sinovac...",7461932.0,16.72,6110712.0,13.70,30.72,13704895.0,30.236599,115.141146,13.481836
Andorra,3,Europe,42156,153.0,41021.0,982.0,14.0,543983,1974.0,249838.0,...,"Moderna, Oxford/AstraZeneca, Pfizer/BioNTech",57817.0,74.74,53367.0,68.99,196.50,151997.0,196.137815,16.873334,68.865088
Angola,4,Africa,99194,1900.0,97149.0,145.0,NaN,2853,55.0,1499795.0,...,Oxford/AstraZeneca,11235059.0,33.11,5993792.0,17.66,51.68,17535411.0,50.433637,6.613837,17.238759


In [23]:
# create new csv file
summary.to_csv('C:/Users/FooJacky/Documents/2022 Sem 2/FIT3179/FIT3179_Assignment2/data/summary_and_vaccine_data.csv')